### Implementing a ChatAgentExector with Langgraph

The agent executor takes a list of messages as input and outputs a list of messages. All agent state is represented as a list of messages. This specifically uses OpenAI function calling. This is the recommended agent executor for newer chat based models that support function calling


In [1]:
import os 
import getpass
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

### Setting up the tools 

When building an agent based application the first step is to determine what tools would be used by the agents, and either make use of prebuilt langchain tools or define our own tools.

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [4]:
# our agent would only be making use of the tavily searct tool
tools = [TavilySearchResults(max_results=2)]

We can now wrap these tools in a simple tool executor. This is a real simple class that takes ina  tool invocation (action) and calls that tool, returning the output. The Tool invocation is any class with tool and tool_input attribute. This describes the function we were looking for earlier which takes in the object dict produced by the language model and calls the right tool and passes in the relavant paremeters while handling any other failures or exceptions, this makes the operation of openai functions much easier. 

In [5]:
from langgraph.prebuilt import ToolExecutor

# next we define a tool executor which would be used for executing those actions
tool_executor = ToolExecutor(tools=tools)

#### Setting up the model 
We would be using the mixtral 8x7B instruct v0.1 model for our experiments, we would be making use of it though the ChatOpenAI class provided by langchain, we would be enabling streaming on the model. The model should also have function calling ability as the nature of langgraph depends on the function calling capability of the model(most language based agetnt applications do)

In [6]:
from langchain_openai.chat_models import ChatOpenAI
model = ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1") # streaming doesn't work well with function calling for mistral models or other opensource models.

Next, we convert our defined tools into openai function tools so that they can be binded to our model

In [7]:
from langchain.tools.render import format_tool_to_openai_function, format_tool_to_openai_tool

In [8]:
functions = [format_tool_to_openai_function(tool) for tool in tools]

In [9]:
from pprint import pprint

In [10]:
pprint(functions)

[{'description': 'A search engine optimized for comprehensive, accurate, and '
                 'trusted results. Useful for when you need to answer '
                 'questions about current events. Input should be a search '
                 'query.',
  'name': 'tavily_search_results_json',
  'parameters': {'properties': {'query': {'description': 'search query to look '
                                                         'up',
                                          'type': 'string'}},
                 'required': ['query'],
                 'type': 'object'}}]


In [11]:
model = model.bind_functions(functions=functions)

Now we have created the tools and bound them to our model. Our model can bind to both functions and tools, which is weird since we are not sure which is the right behaviour, but this is somthing definitely worth investigating.

### Defining the agent state
There are various types of langgraphs, the main type of langgraph is the StatefulGraph. The graph is parameterized by a state object that is passed around each node and conditional edge. Each nod thne returns operations to update the state. The operations can either SET specific attributes on the state or ADD to the existing attribute. whether to set or ad is denoted by annoting the state object you construct the graph with. 

For this example we would be using a simple TypeDict to describe our state, it would contain a field for messages which would store the previous message stream. 

In [12]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

In [13]:
class AgentState(TypedDict):
    '''
    Annotation allows us to define annotated types. This can allow us provide more relevant context to the type of our variable. 
    Here are some examples of how annotated can be used. 
        from typing import Annotated

        # Define a variable with additional metadata
        x: Annotated[int, "This is an integer"] = 5

        # Define a function signature with annotated parameters
        def greet(name: Annotated[str, "Name of the person"]):
            print(f"Hello, {name}!")

        # Using the function with annotated parameters
        greet("Alice")

    '''
    messages: Annotated[Sequence[BaseMessage], operator.add] 

### Define the nodes within the graph
The operating work horses of our graph are the nodes, they execute some type of computation or runnable, a function, agent, chain or something else depending on the context of the application. 

In [14]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage


def should_continue(state: AgentState):
    """
    Determines what should be the next operation after model call on the chain entry point
    """
    messages = state["messages"]
    last_message: BaseMessage = messages[-1]
    
    if "tool_calls" not in last_message.additional_kwargs:
        return "end"
    else:
        return "continue"
    
    
def call_model(state: AgentState):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}



def call_tool(state: AgentState):
    messages = state["messages"]
    last_message: BaseMessage = messages[-1]
    fn = last_message.additional_kwargs["tool_calls"][0]["function"]
    action = ToolInvocation(
        tool=fn["name"],
        tool_input=json.loads(fn["arguments"])
    )
    
    response = tool_executor.invoke(action)
    function_message = FunctionMessage(content=str(response), name=action.tool)
    return {"messages": [function_message]}

### Defining the graph
Here we difne the high level architecture of the overall langraph graph, we would be adding all the nodes to the workflow and defining the conditional edges for the operation of our graph. Finally, we would be compiling our graph into an app

In [15]:
from langgraph.graph import StateGraph,  END 


workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

workflow.add_conditional_edges("agent", should_continue, {"continue": "action", "end": END})
workflow.set_entry_point("agent")
workflow.add_edge("action", "agent")

app = workflow.compile()

In [16]:
pprint(app.to_json())

{'id': ['langgraph', 'pregel', 'Pregel'],
 'lc': 1,
 'repr': "Pregel(nodes={'agent': ChannelInvoke(bound=RunnableLambda(...)\n"
         '| RunnableLambda(call_model)\n'
         "| ChannelWrite(channels=[('agent', None, False), ('messages', "
         "RunnableLambda(...), False)]), config={'tags': []}, channels={None: "
         "'agent:inbox'}, triggers=['agent:inbox']), 'action': "
         'ChannelInvoke(bound=RunnableLambda(...)\n'
         '| RunnableLambda(call_tool)\n'
         "| ChannelWrite(channels=[('action', None, False), ('messages', "
         "RunnableLambda(...), False)]), config={'tags': []}, channels={None: "
         "'action:inbox'}, triggers=['action:inbox']), 'agent:edges': "
         'ChannelInvoke(bound=RunnableLambda(_read)\n'
         "| RunnableLambda(runnable), config={'tags': ['langsmith:hidden']}, "
         "channels={None: 'agent'}, triggers=['agent']), 'action:edges': "
         'ChannelInvoke(bound=RunnableLambda(_read)\n'
         "| ChannelWrite(c

### Using the graph
Now that we have built and compiled the graph we can now make use of it.

In [17]:
from langchain_core.messages import HumanMessage

In [18]:
inputs = {
    "messages": [HumanMessage(content="what is the weather in SF?")]
}
app.invoke(inputs)

{'messages': [HumanMessage(content='what is the weather in SF?'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9ofd0so5kkjw31bwxfo1yen9', 'function': {'arguments': '{"query":"weather in SF"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}),
  FunctionMessage(content="[{'url': 'https://www.wunderground.com/forecast/us/ca/san-francisco', 'content': 'Get the latest weather forecast for San Francisco, CA from Weather Underground, including current conditions, hourly and 10-day forecasts, maps and radar. See the chance of rain, snow, cloud cover, humidity, pressure and more for your location.'}, {'url': 'https://www.accuweather.com/en/us/san-francisco/94103/weather-forecast/347629', 'content': 'San Francisco, CA Weather Forecast | AccuWeather Daily Radar Monthly Coastal Flood Advisory Current Weather 3:28 AM 48° F RealFeel® 50° Air Quality Poor Wind NNE 2 mph Wind Gusts 3 mph...'}]", name='tavily_search_results_json'),
  AIMessage(content='', additio

##### Streaming the outputs
We can stream the outputs coming from the large language model

In [19]:
inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zndfq5la4459kqmha8jnf7ng', 'function': {'arguments': '{"query":"sf weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]})]}

---

Output from node 'action':
---
{'messages': [FunctionMessage(content='[{\'url\': \'https://forecast.weather.gov/zipcity.php?inputstring=San+Francisco,CA\', \'content\': \'NOAA National Weather Service National Weather Service. Toggle navigation. HOME; FORECAST . Local; Graphical; Aviation; Marine; Rivers and Lakes; Hurricanes; Severe Weather; Fire Weather; ... San Francisco CA 37.77°N 122.41°W (Elev. 131 ft) Last Update: 12:26 am PST Feb 14, 2024. Forecast Valid: 2am PST Feb 14, 2024-6pm PST Feb 20, 2024 .\'}, {\'url\': \'https://weather.com/weather/hourbyhour/l/USCA0987:1:US\', \'content\': "recents\\nSpecialty Forecasts\\nHourly Weather-San Francisco, CA\\nBeach Hazard Statement\\nSaturday, November 25\\n5 pm\\nClear\\n6 pm\\n